# COURSE: Intelligent information systems
## LAB 04

In [1]:
import string

import numpy as np
import pandas as pd
from nltk import WordNetLemmatizer, pos_tag, RegexpParser
from nltk.corpus import words, stopwords, sentiwordnet , wordnet, movie_reviews
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

from keras.layers import Embedding
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.layers import Input

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
movie_categories = movie_reviews.categories()
movie_ids = movie_reviews.fileids()
single_movie_words = movie_reviews.words(movie_ids[0])
english_stopwords = stopwords.words('english')

text = """Stemming some multi sentence.
Text"""
sentences = sent_tokenize(text)
words_all = word_tokenize(text)
WORD_TAG_LST = pos_tag(single_movie_words)
print("List of tuples (word, POS_TAG):", WORD_TAG_LST)
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
# print(movie_categories)
print(movie_ids)
print("Single movie words", single_movie_words)

wrd = stemmer.stem(words_all[0])
print("Stem res: ", words_all[0], wrd)

wrd_lem = lemmatizer.lemmatize(WORD_TAG_LST[0][0], pos="n")
print("Lem res:", WORD_TAG_LST[0][0], wrd_lem)

List of tuples (word, POS_TAG): [('plot', 'NN'), (':', ':'), ('two', 'CD'), ('teen', 'NN'), ('couples', 'NNS'), ('go', 'VBP'), ('to', 'TO'), ('a', 'DT'), ('church', 'NN'), ('party', 'NN'), (',', ','), ('drink', 'NN'), ('and', 'CC'), ('then', 'RB'), ('drive', 'NN'), ('.', '.'), ('they', 'PRP'), ('get', 'VBP'), ('into', 'IN'), ('an', 'DT'), ('accident', 'NN'), ('.', '.'), ('one', 'CD'), ('of', 'IN'), ('the', 'DT'), ('guys', 'NN'), ('dies', 'VBZ'), (',', ','), ('but', 'CC'), ('his', 'PRP$'), ('girlfriend', 'NN'), ('continues', 'VBZ'), ('to', 'TO'), ('see', 'VB'), ('him', 'PRP'), ('in', 'IN'), ('her', 'PRP$'), ('life', 'NN'), (',', ','), ('and', 'CC'), ('has', 'VBZ'), ('nightmares', 'NNS'), ('.', '.'), ('what', 'WP'), ("'", "''"), ('s', 'VBD'), ('the', 'DT'), ('deal', 'NN'), ('?', '.'), ('watch', 'VB'), ('the', 'DT'), ('movie', 'NN'), ('and', 'CC'), ('"', 'NNP'), ('sorta', 'VBP'), ('"', 'NNP'), ('find', 'VBP'), ('out', 'RP'), ('.', '.'), ('.', '.'), ('.', '.'), ('critique', 'NN'), (':', ':

### Dataset import

In [3]:
#EMOTION LVL  0 : NO , 1: LOW , 2: MODERATE, 3: HIGH

ElocEN_anger = pd.read_table("data/2018-ELocEN/2018-EI-oc-En-anger-dev.txt")
ElocEN_fear = pd.read_table("data/2018-ELocEN/2018-EI-oc-En-fear-dev.txt")
ElocEN_joy = pd.read_table("data/2018-ELocEN/2018-EI-oc-En-joy-dev.txt")
ElocEN_sadness = pd.read_table("data/2018-ELocEN/2018-EI-oc-En-sadness-dev.txt")

# Collect all
frames = [ElocEN_sadness, ElocEN_anger, ElocEN_joy, ElocEN_fear]
ElocEN_full = pd.concat(frames, ignore_index=True)
print(ElocEN_full["Intensity Class"])
ElocEN_full["Intensity Class Value"] = ElocEN_full["Intensity Class"].apply(lambda x: int(x[0]))
print(ElocEN_full["Intensity Class Value"])

0                       0: no sadness can be inferred
1                       0: no sadness can be inferred
2       2: moderate amount of sadness can be inferred
3           3: high amount of sadness can be inferred
4                       0: no sadness can be inferred
                            ...                      
1459                       0: no fear can be inferred
1460                       0: no fear can be inferred
1461                       0: no fear can be inferred
1462                       0: no fear can be inferred
1463                       0: no fear can be inferred
Name: Intensity Class, Length: 1464, dtype: object
0       0
1       0
2       2
3       3
4       0
       ..
1459    0
1460    0
1461    0
1462    0
1463    0
Name: Intensity Class Value, Length: 1464, dtype: int64


In [4]:
# NLP

# Replace emojis with attributes. EX: :) -> smile , :( -> sad
english_stopwords = stopwords.words('english')

import re
import emoji
from collections import Counter
def remove_stopwords(text):
    text_words = word_tokenize(text)
    clean_words = []
    for word in text_words:
        if word not in english_stopwords:
            clean_words.append(word)
    # clean_sent.append(" ".join(clean_words))
    # clean_sent.append(clean_words)
    return " ".join(clean_words)
def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)
def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+','', text) # remove @ mentions
    text = re.sub(r'#','', text) # remove tags
    text = re.sub(r'RT[\s]+','', text) # remove RT
    text = re.sub(r'https?:\/\/\S+','', text) # remove links
    text = give_emoji_free_text(text)
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
        )
    text = emoji_pattern.sub(r"", text)
    table = str.maketrans("", "", string.punctuation)
    text = text.translate(table)
    return text
def perform_stemming(text):
    stem_wrds = []
    for wrd in text.split(" "):
        stem_wrds.append(stemmer.stem(wrd))
    return " ".join(stem_wrds)
def perform_lemmatization(text):
    lem_wrds = []
    for wrd in text.split(" "):
        lem_wrds.append(lemmatizer.lemmatize(wrd))
    return " ".join(lem_wrds)
def perform_lemmatization_POSTAG(wrd_list):
    lem_wrds = []
    for wrd, pos_tg in wrd_list:
        p_tag = pos_tg[0].lower()
        if p_tag in ['n','a','r','v']:
            lem_wrds.append(lemmatizer.lemmatize(wrd, p_tag))
        else:
            lem_wrds.append(lemmatizer.lemmatize(wrd))
    return " ".join(lem_wrds)
def perform_tagging(text):
    tag = pos_tag(text.split(" "))
    return tag
chunkGram = r"""Chunk: {<.*>+}
                        }<IN|DT|TO>+{""" # CHUNKING AND CHINKING
chunkParser = RegexpParser(chunkGram)

def perform_chunking(wrd_lst):
    return chunkParser.parse(wrd_lst)


def word_counter(tweets):
    count = Counter()
    size_count = []
    for tweet in tweets:
        total_count = 0
        wrds = tweet.split(" ")
        total_count = len(wrds)
        size_count.append(total_count)
        for wrd in wrds:
            count[wrd] += 1
    return count, size_count
def value_classes(dimension):
    if dimension == "sadness":
        return 0
    if dimension == "anger":
        return 1
    if dimension == "fear":
        return 2
    if dimension == "joy":
        return 3

In [5]:
ElocEN_full["Tweet_clean"] = ElocEN_full["Tweet"].apply(clean_text)
ElocEN_full["Tweet_clean"] = ElocEN_full["Tweet_clean"].apply(remove_stopwords)
# ElocEN_full["Tweet_sent_tokenize"] = ElocEN_full["Tweet"].apply(lambda x: sent_tokenize(x))
# print(ElocEN_full["Tweet_sent_tokenize"][15])
print(ElocEN_full["Tweet_clean"].head(8))
ElocEN_full["Tweet_stemmed"] = ElocEN_full["Tweet_clean"].apply(perform_stemming)
ElocEN_full["Tweet_tagged"] = ElocEN_full["Tweet_clean"].apply(perform_tagging)
ElocEN_full["Tweet_lemmed"] = ElocEN_full["Tweet_clean"].apply(perform_lemmatization)
ElocEN_full["Tweet_lemmed_POSTAG"] = ElocEN_full["Tweet_tagged"].apply(perform_lemmatization_POSTAG)
ElocEN_full["Tweet_chunked"] = ElocEN_full["Tweet_tagged"].apply(perform_chunking)

print(ElocEN_full["Tweet_stemmed"].head(2))
print(ElocEN_full["Tweet_tagged"].head(2))
print(ElocEN_full["Tweet_lemmed_POSTAG"].head(3))
print(ElocEN_full["Tweet_chunked"].head(2))

# classes to float values
ElocEN_full["Dimension_int"] = ElocEN_full["Affect Dimension"].apply(value_classes)
print(ElocEN_full["Dimension_int"])
ElocEN_full["Classes"] = ElocEN_full["Dimension_int"] + (ElocEN_full["Intensity Class Value"].apply(lambda x: x/10))
print(ElocEN_full["Classes"])

0    We opportunity move away gloomy nihilism chara...
1    I know fact theyll either ignore fact tiff isn...
2    A religion pushes people suicide punishes righ...
3    You could hundred million followers still genu...
4                         Isnt always control mistrust
5    Went bed 130 fell asleep niece started crying ...
6                                   Very sad upset idk
7                  shouldve cut moment started hurting
Name: Tweet_clean, dtype: object
0    we opportun move away gloomi nihil character r...
1    i know fact theyll either ignor fact tiff isnt...
Name: Tweet_stemmed, dtype: object
0    [(We, PRP), (opportunity, NN), (move, VB), (aw...
1    [(I, PRP), (know, VBP), (fact, NN), (theyll, I...
Name: Tweet_tagged, dtype: object
0    We opportunity move away gloomy nihilism chara...
1    I know fact theyll either ignore fact tiff isn...
2    A religion push people suicide punishes right ...
Name: Tweet_lemmed_POSTAG, dtype: object
0    [[(We, PRP), (opportunity, 

In [6]:
# Preparing dataset
counter, lst_sizes = word_counter(list(ElocEN_full["Tweet_lemmed_POSTAG"]))
num_words = len(counter)
print("Total words in the bag: ", num_words)
print(counter)
bagOfWords = set(counter.keys())
print(bagOfWords)
print("Max seq. size:", np.max(lst_sizes))

Total words in the bag:  3923
Counter({'I': 443, 'get': 114, 'Im': 104, 'like': 92, 'amp': 82, 'u': 71, 'make': 70, 'go': 69, 'start': 68, 'day': 65, 'dont': 62, 'people': 61, 'time': 55, 'The': 50, 'need': 50, 'back': 49, 'love': 48, 'think': 45, 'one': 44, 'today': 43, 'know': 42, 'see': 41, 'sad': 40, 'feel': 40, 'happy': 40, 'thing': 39, 'never': 39, 'smile': 39, 'want': 36, 'life': 36, 'work': 34, 'take': 34, 'look': 34, 'give': 32, 'If': 32, 'new': 32, 'call': 31, 'would': 31, 'You': 30, 'good': 30, 'lose': 29, 'It': 29, 'So': 28, 'laugh': 28, 'friend': 28, 'really': 28, '’': 28, 'ever': 28, 'dark': 27, 'bad': 27, 'fear': 27, 'depress': 27, 'leave': 27, 'even': 26, 'How': 26, 'na': 26, 'well': 26, 'great': 26, 'cry': 25, 'try': 25, 'someone': 25, 'come': 25, 'This': 24, 'What': 24, 'That': 24, 'horrible': 23, 'When': 23, 'help': 23, 'watch': 23, 'Ive': 23, 'hope': 23, 'do': 23, 'A': 22, 'still': 22, 'dreadful': 22, 'Not': 22, 'mad': 22, 'something': 22, 'live': 22, 'annoy': 22, '

In [7]:
tweetTrain, tweetTest = train_test_split(ElocEN_full, train_size=0.8, stratify=ElocEN_full["Affect Dimension"])
tweetVal, tweetTest = train_test_split(tweetTest, train_size=0.5, stratify=tweetTest["Affect Dimension"])
print("Train size", len(tweetTrain), "Test size: ", len(tweetTest), "Val size: ", len(tweetVal))

print(wordnet)

Train size 1171 Test size:  147 Val size:  146
<WordNetCorpusReader in 'C:\\Users\\HS-Rn7\\AppData\\Roaming\\nltk_data\\corpora\\wordnet'>


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(list(tweetTrain["Tweet_lemmed_POSTAG"]))
wordIndex = tokenizer.word_index
print(wordIndex)
print(len(wordIndex))
max_size = np.max(lst_sizes)

{'i': 1, 'get': 2, 'im': 3, 'like': 4, 'amp': 5, 'go': 6, 'make': 7, 'dont': 8, 'start': 9, 'people': 10, 'time': 11, 'u': 12, 'day': 13, 'today': 14, 'one': 15, 'back': 16, 'happy': 17, 'need': 18, 'think': 19, 'never': 20, 'the': 21, 'sad': 22, 'cant': 23, 'love': 24, 'see': 25, 'know': 26, 'feel': 27, 'good': 28, 'life': 29, 'you': 30, 'look': 31, 'thing': 32, 'want': 33, 'smile': 34, 'ever': 35, 'great': 36, 'it': 37, 'would': 38, 'new': 39, '’': 40, 'call': 41, 'do': 42, 'give': 43, 'really': 44, 'take': 45, 'even': 46, 'laugh': 47, 'if': 48, 'work': 49, 'fear': 50, 'bad': 51, 'try': 52, 'not': 53, 'lose': 54, 'well': 55, 'this': 56, 'ive': 57, 'friend': 58, 'horrible': 59, 'dark': 60, 'na': 61, 'what': 62, 'leave': 63, 'someone': 64, 'depress': 65, 'that': 66, 'when': 67, 'come': 68, 'so': 69, 'worry': 70, 'youre': 71, 'best': 72, 'beautiful': 73, 'keep': 74, 'still': 75, 'shit': 76, 'live': 77, 'show': 78, 'thats': 79, 'next': 80, 'hurt': 81, 'annoy': 82, 'something': 83, 'alway

In [9]:
train_sequences = tokenizer.texts_to_sequences(list(tweetTrain["Tweet_lemmed_POSTAG"]))
test_sequences = tokenizer.texts_to_sequences(list(tweetTest["Tweet_lemmed_POSTAG"]))
val_sequences = tokenizer.texts_to_sequences(list(tweetVal["Tweet_lemmed_POSTAG"]))
train_padded_sequences = pad_sequences(train_sequences, max_size, padding="post", truncating="post")
test_padded_sequences = pad_sequences(test_sequences, max_size, padding="post", truncating="post")
val_padded_sequences = pad_sequences(val_sequences, max_size, padding="post", truncating="post")  # seq = list of sentences that include a list of words

reverse_word_index = dict([(value, key) for (key,value) in wordIndex.items()])
def decode(text):
    return " ".join([reverse_word_index.get(i, "?") for i in text])
print(list(tweetTrain["Tweet_lemmed_POSTAG"])[0])
print(train_sequences[0])
print(decode(train_sequences[0]))
print(train_padded_sequences[0])

revenge Change You ChooseASide country India
[229, 169, 30, 1766, 536, 371]
revenge change you chooseaside country india
[ 229  169   30 1766  536  371    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]


## Zad 1 (LSTM)

In [10]:
import os
import _pickle as pickle
import numpy as np

def load_embeddings(file_name, vocabulary):
    """
    Loads word embeddings from the file with the given name.
    :param file_name: name of the file containing word embeddings
    :type file_name: str
    :param vocabulary: captions vocabulary
    :type vocabulary: numpy.array
    :return: word embeddings
    :rtype: dict
    """
    embeddings = dict()
    with open(file_name, 'r', encoding='utf-8') as doc:
        line = doc.readline()
        while line != '':
            line = line.rstrip('\n').lower()
            parts = line.split(' ')
            vals = np.array(parts[1:], dtype=np.float)
            if parts[0] in vocabulary:
                embeddings[parts[0]] = vals
            line = doc.readline()
    return embeddings

def load_embedding_weights_new(vocabulary, embedding_size, embeddings=None):
    """ Creates and loads embedding weights.
    :param embeddings: preloaded embeddings( specific for the bagOfWords)
    :param vocabulary: captions vocabulary
    :type vocabulary: numpy.array
    :param embedding_size: embedding size :type embedding_size: int :return: embedding weights
    :rtype: numpy.array """
    if os.path.exists('data/embedding_matrix.pkl'):
        with open('data/embedding_matrix.pkl', 'rb') as f:
            embedding_matrix = pickle.load(f)
    else:
        print('Creating embedding weights...')
        if embeddings is None:
            embeddings = load_embeddings(f'data/GloVe/glove.6B.{embedding_size}d.txt', vocabulary)
        embedding_matrix = np.zeros((len(vocabulary), embedding_size))
        for i in range(len(vocabulary)):
            if vocabulary[i] in embeddings.keys():
                embedding_matrix[i] = embeddings[vocabulary[i]]
            else:
                embedding_matrix[i] = np.random.standard_normal(embedding_size)
        with open('data/embedding_matrix.pkl', 'wb') as f:
            pickle.dump(embedding_matrix, f)
    return embedding_matrix
def load_embeddings_custom(vocabulary, emb_size, embeddings=None):
    if os.path.exists('data/embedding_matrix.pkl'):
        with open('data/embedding_matrix.pkl', 'rb') as f:
            embedding_matrix = pickle.load(f)
    else:
        print('Creating embedding weights...')
        if embeddings is None:
            embeddings = load_embeddings(f'data/GloVe/glove.6B.{emb_size}d.txt', vocabulary)
        num_wrds = len(vocabulary) + 1
        embedding_matrix = np.zeros((num_wrds, emb_size))
        for word, i in vocabulary.items():
            if i < num_wrds:
                emb_vec = embeddings.get(word)
                if emb_vec is not None:
                    embedding_matrix[i] = emb_vec
                else:
                    embedding_matrix[i] = np.random.standard_normal(emb_size)
        with open('data/embedding_matrix.pkl', 'wb') as f:
            pickle.dump(embedding_matrix, f)
    return embedding_matrix

In [11]:
# from Scripts.Lab04 import context_shifting, dependency_parsing, eval_generated_text, lemmatization
from Scripts.Lab04 import visualize, word_embeddings
from Scripts.Lab04.eval_metrics.bleu import bleu, bleu_scorer
from Scripts.Lab04.eval_metrics.rouge import rouge
from Scripts.Lab04.eval_metrics.tokenizer import ptbtokenizer
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

In [12]:
# glove import dictionary
GloVe_embeddings = load_embeddings("data/GloVe/glove.6B.50d.txt",list(wordIndex.keys()))

In [13]:
list(bagOfWords)

['Rooney',
 'station',
 'homemade',
 'RIPBiwott',
 'PTX',
 'greatRafa',
 'worship',
 'Each',
 'Wtf',
 'Doom',
 'persevere',
 'George',
 'Tolle',
 'C130',
 'London',
 'PARX',
 'Broncos',
 'youve',
 'wolf',
 'awful',
 'grandma',
 'demon',
 'rude',
 'problem',
 'fantastic',
 'Obviously',
 'glioblastoma',
 'hatersgonnahate',
 'Past',
 'bank',
 'Rio',
 'softly',
 'FREE',
 'despite',
 'account',
 'free',
 'SINCE',
 'acre',
 'lol',
 'smaller',
 'fast',
 'carpet',
 'Bastard',
 'RadicalIslam',
 'currentfeels',
 'How',
 'tomorrow',
 'Novak',
 'scarcity',
 'RHOC',
 'skid',
 'AllStarGame2017',
 'alla',
 'freak',
 'Saleem',
 'Apologies',
 'offer',
 '93lee',
 'congratulation',
 'primer',
 'pursue',
 'Man',
 'still',
 'hook',
 'tiff',
 'patriarchal',
 'firstIt',
 'Im',
 'CoffeennNow',
 'wrong',
 'regular',
 'potato',
 'assistant',
 'wattpad',
 'wed',
 'country',
 'food',
 'cubicle',
 'magpie',
 'Thought',
 'miss',
 'join',
 'episode',
 'yeri',
 'NEVER',
 'fest',
 'Effect',
 'birthday',
 'nice',
 'sho

In [14]:
# np array (matrix)
GloVe_weights = load_embeddings_custom(wordIndex, 50, GloVe_embeddings)
print(GloVe_weights[20])
print(wordIndex["today"])
print(GloVe_embeddings.get("today"))

[ 2.7751e-04  4.2673e-01 -8.2938e-02  2.7601e-01  6.4721e-01 -9.1728e-01
 -6.3471e-01 -2.8023e-01 -6.6653e-01 -2.8436e-01 -6.4249e-02 -4.3626e-01
 -1.0830e-01 -3.5818e-01  7.2311e-01  6.5368e-01 -2.9573e-01  1.2007e-01
 -2.9959e-02 -2.0594e-01  2.0017e-01  1.6421e-01  1.5202e-01 -2.4855e-02
  5.2887e-01 -1.3625e+00 -5.6036e-01  1.7777e-01 -9.1003e-02  9.7549e-02
  3.5102e+00  1.0631e-01  6.5602e-02 -8.0777e-02 -1.2553e-01 -6.9932e-01
 -1.5068e-02  3.9353e-01 -2.8195e-03  2.0635e-01 -4.7726e-01 -1.2639e-01
  2.9399e-01  1.0000e-01  3.4015e-04  6.2769e-01 -4.5344e-01  3.9615e-01
  1.8857e-02  1.7536e-01]
14
[ 2.7751e-04  4.2673e-01 -8.2938e-02  2.7601e-01  6.4721e-01 -9.1728e-01
 -6.3471e-01 -2.8023e-01 -6.6653e-01 -2.8436e-01 -6.4249e-02 -4.3626e-01
 -1.0830e-01 -3.5818e-01  7.2311e-01  6.5368e-01 -2.9573e-01  1.2007e-01
 -2.9959e-02 -2.0594e-01  2.0017e-01  1.6421e-01  1.5202e-01 -2.4855e-02
  5.2887e-01 -1.3625e+00 -5.6036e-01  1.7777e-01 -9.1003e-02  9.7549e-02
  3.5102e+00  1.0631e-

In [15]:
(GloVe_weights[20] == GloVe_embeddings.get("today")).all()

True

In [19]:
model = Sequential()
model.add(Embedding(num_words + 1, 50, embeddings_initializer=Constant(GloVe_weights), input_length=max_size, trainable=False)) # experiment with n_outputs
model.add(LSTM(50, dropout=0.1))
model.add(Dense(1, activation="sigmoid"))
optimizer = Adam(lr=3e-4)

model.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=["accuracy"])

history = model.fit(
    train_padded_sequences, list(tweetTrain["Dimension_int"]),
    epochs=20, validation_data=(val_padded_sequences, list(tweetVal["Dimension_int"])),
    verbose=1
)
s_len = max_size

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 1171 samples, validate on 146 samples
Epoch 1/20
1171/1171 [==============================] - 1s - loss: 0.5852 - acc: 0.2622 - val_loss: 0.3975 - val_acc: 0.2671
Epoch 2/20
1171/1171 [==============================] - 0s - loss: -0.6406 - acc: 0.2647 - v

## Zad 2

In [21]:
from tensorflow import metrics
from tensorflow_addons.metrics import F1Score
from tensorflow_addons.optimizers import AdamW
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
for training_tweet in tweetTrain["Tweet_lemmed_POSTAG"]:
    bert_tokenizer.encode_plus(training_tweet, add_special_tokens=True,
                           max_length=s_len, pad_to_max_length=True, truncation=True)

input_ids = Input(shape=(s_len,), name='input_token', dtype='int32')
att_masks = Input(shape=(s_len,), name='masked_token', dtype='int32')
bert_in = bert_model(input_ids, attention_mask=att_masks)

keras_bert = Sequential()
keras_bert.add(bert_in)
keras_bert.add(Dense(1, activation="softmax"))

optimizer_adamw = AdamW(learning_rate=3e-5)
# optimizer_adamw = optimization.create_optimizer

metrics = [metrics.Accuracy, metrics.Recall, metrics.Precision, F1Score]
keras_bert.compile(optimizer= optimizer_adamw,
                   loss="categorical_crossentropy",
                   metrics=metrics)
# encoded_input = tokenizer(text, return_tensors='tf')
# output = bert_model(encoded_input)
history = keras_bert.fit(train_padded_sequences, list(tweetTrain["Dimension_int"]),
    epochs=20, validation_data=(val_padded_sequences, list(tweetVal["Dimension_int"])),
    verbose=1)

ValueError: Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.